<a href="https://colab.research.google.com/github/garodisk/Text-Mining-using-word2vec-distributed-representation/blob/master/Transfer_Learning_without_extra_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UMICH SI650 - Sentiment Classification**

This is a text classification task - sentiment classification. Every document (a line in the data file) is a sentence extracted from social media (blogs). Your goal is to classify the sentiment of each sentence into "positive" or "negative".

The training data contains 7086 sentences, already labeled with 1 (positive sentiment) or 0 (negative sentiment). The test data contains 33052 sentences that are unlabeled. The submission should be a .txt file with 33052 lines. In each line, there should be exactly one integer, 0 or 1, according to your classification results.

I will use 3 approaches for this:

**Approach 1** : Just using google's word2vec weights built on google news

**Approach 2** : Fine Tune the weights from google using some extra convolutional layers

**Approach 3** : Just using deep learning without pre-trained weights

This notebook is on approach 1


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

In [0]:
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
#from make_tensorboard import make_tensorboard
import codecs

Using TensorFlow backend.


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
pip install tensorboardcolab


In [0]:
#using tensorboard to display learning graph

from tensorboardcolab import *
tbc=TensorBoardColab()
tbc=TensorBoardColab(startup_waiting_time=30)


Wait for 8 seconds...
TensorBoard link:
https://02dd3343.ngrok.io
Wait for 30 seconds...
TensorBoard link:
https://02dd3343.ngrok.io


In [0]:
from google.colab import files
uploaded = files.upload()

Saving training.txt to training.txt


In [0]:
np.random.seed(42)

INPUT_FILE = "training.txt"
VOCAB_SIZE = 5000
EMBED_SIZE = 300
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [0]:
print("reading data...")
counter = collections.Counter()
fin = codecs.open(INPUT_FILE, "r", encoding='utf-8')
maxlen = 0
for line in fin:
    _, sent = line.strip().split("\t")
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1
fin.close()

reading data...


In [0]:
print("creating vocabulary...")
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}
index2word[0] = "_UNK_"


creating vocabulary...


In [0]:
print("creating word sequences...")
ws, ys = [], []
fin = codecs.open(INPUT_FILE, "r", encoding='utf-8')
for line in fin:
    label, sent = line.strip().split("\t")
    ys.append(int(label))
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    wids = [word2index[word] for word in words]
    ws.append(wids)
fin.close()
W = pad_sequences(ws, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

creating word sequences...


In [0]:
# load word2vec model using gensim library

from gensim.models import KeyedVectors
pretrained_embeddings_path = "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


word2vec = KeyedVectors.load_word2vec_format(pretrained_embeddings_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
print("transferring embeddings...")
X = np.zeros((W.shape[0], EMBED_SIZE))
for i in range(W.shape[0]):
    E = np.zeros((EMBED_SIZE, maxlen))
    words = [index2word[wid] for wid in W[i].tolist()]
    for j in range(maxlen):
        try:
            E[:, j] = word2vec[words[j]]
        except KeyError:
            pass
    X[i, :] = np.sum(E, axis=1)

transferring embeddings...


In [0]:
Xtrain, Xtest, Ytrain, Ytest = \
    train_test_split(X, Y, test_size=0.3, random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

(4960, 300) (2126, 300) (4960, 2) (2126, 2)


In [0]:
from keras.layers.core import Dense, Dropout

model = Sequential()
model.add(Dense(32, input_dim=EMBED_SIZE, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[TensorBoardColabCallback(tbc)],
                    validation_data=(Xtest, Ytest))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4960 samples, validate on 2126 samples







Epoch 1/10
4960/4960 [==============================] - 1s 106us/step - loss: 0.4553 - acc: 0.8198 - val_loss: 0.2141 - val_acc: 0.9520

Epoch 2/10
4960/4960 [==============================] - 0s 36us/step - loss: 0.1525 - acc: 0.9615 - val_loss: 0.1062 - val_acc: 0.9690
Epoch 3/10
4960/4960 [==============================] - 0s 33us/step - loss: 0.0877 - acc: 0.9756 - val_loss: 0.0861 - val_acc: 0.9812
Epoch 4/10
4960/4960 [==============================] - 0s 37us/step - loss: 0.0650 - acc: 0.9810 - val_loss: 0.0726 - val_acc: 0.9859
Epoch 5/10
4960/4960 [==============================] - 0s 35us/step - loss: 0.0531 - acc: 0.9839 - val_loss: 0.0643 - val_acc: 0.9840
Epoch 6/10
4960/4960 [==============================] - 0s 38us/step - loss: 0.0462 - acc: 0.9881 - val_loss: 0.0585 - val_acc: 0.9840
Epoch 7/10
4960/4960 [==============

In [0]:
# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

2126/2126 [==============================] - 0s 25us/step
Test score: 0.053, accuracy: 0.985
